In [7]:
# Run setup from config notebook
%run 0_config_setup.ipynb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


GPU Available: True
GPU Name: NVIDIA GeForce RTX 5090
GPU Memory: 33.67 GB
GPU Available: True
GPU Name: NVIDIA GeForce RTX 5090
GPU Memory: 33.67 GB
Using virtual environment at: /home/imane/gemma_env
Hugging Face token loaded from cache.
Directory structure and model paths are ready!
Configuration loaded successfully!
Utility functions loaded!
Configuration saved to /home/imane/Downloads/Reinforcement Learning/config.json
Using virtual environment at: /home/imane/gemma_env
Hugging Face token loaded from cache.
Directory structure and model paths are ready!
Configuration loaded successfully!
Utility functions loaded!
Configuration saved to /home/imane/Downloads/Reinforcement Learning/config.json


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import json
import itertools
import random

set_seed(SEED)

# ===========================
# COMET Model Setup (for quality scoring)
# ===========================
# Using COMET-QE (Quality Estimation) - no reference needed
# Install: pip install unbabel-comet

try:
    from comet import download_model, load_from_checkpoint
    
    # Download COMET-QE model (reference-free quality estimation)
    # Options: "Unbabel/wmt22-cometkiwi-da" (QE), "Unbabel/wmt22-comet-da" (with ref)
    COMET_MODEL_NAME = "Unbabel/wmt22-cometkiwi-da"  # QE model - no reference needed
    
    print(f"📥 Downloading COMET model: {COMET_MODEL_NAME}")
    comet_model_path = download_model(COMET_MODEL_NAME)
    
    print("🔄 Loading COMET model...")
    comet_model = load_from_checkpoint(comet_model_path)
    
    # Move to GPU if available (use second GPU to keep translation model on first)
    if NUM_GPUS > 1:
        comet_device = 1  # Use GPU 1 for COMET
    elif torch.cuda.is_available():
        comet_device = 0
    else:
        comet_device = "cpu"
    
    # Note: COMET model.predict() handles device internally
    COMET_AVAILABLE = True
    print(f"✅ COMET model loaded successfully")
    print(f"   Model: {COMET_MODEL_NAME}")
    print(f"   Type: Quality Estimation (reference-free)")
    
except ImportError:
    print("⚠️ COMET not installed. Install with: pip install unbabel-comet")
    print("   Will use fallback heuristics for scoring")
    COMET_AVAILABLE = False
    comet_model = None
except Exception as e:
    print(f"⚠️ COMET failed to load: {e}")
    print("   Will use fallback heuristics for scoring")
    COMET_AVAILABLE = False
    comet_model = None

## Load SFT Model

In [ ]:
# Clear GPU cache if needed
import gc
import os

# Set CUDA environment variable for better error reporting
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'  # Set to '1' only for debugging

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("✓ GPU cache cleared")

# ===========================
# HPC OPTIMIZED SETTINGS (2x NVIDIA 5090)
# ===========================
FORCE_CPU = False  # USE GPU on HPC!
USE_BFLOAT16 = True  # bfloat16 is optimal for modern GPUs

print(f"🚀 HPC Mode: Using {NUM_GPUS} GPU(s)")
print(f"   Precision: {'bfloat16' if USE_BFLOAT16 else 'float32'}")
print(f"   Flash Attention: {USE_FLASH_ATTENTION}")

✓ GPU cache cleared
Device mode: CPU (forced)
⚠ Using CPU mode to avoid 'no kernel image' error


In [ ]:
print("Loading SFT model from Hugging Face...")
print("🚀 Optimized for HPC with 2x NVIDIA 5090\n")

# Load directly from Hugging Face Hub
model_name = "ModelSpace/GemmaX2-28-9B-v0.1"

# HPC Optimized loading with multi-GPU support
if FORCE_CPU:
    print("⚠ Loading model on CPU (FORCE_CPU=True)")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map={"": "cpu"},
        torch_dtype=torch.float32,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )
else:
    # Multi-GPU optimized loading
    print(f"Loading model across {NUM_GPUS} GPU(s)...")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    
    # Configure memory allocation per GPU
    if NUM_GPUS > 1:
        max_memory = {i: GPU_MEMORY_PER_DEVICE for i in range(NUM_GPUS)}
        max_memory["cpu"] = "32GB"  # Allow CPU offload if needed
    else:
        max_memory = None
    
    # Load with Flash Attention 2 for faster inference
    model_kwargs = {
        "torch_dtype": torch.bfloat16 if USE_BFLOAT16 else torch.float32,
        "device_map": "auto",
        "trust_remote_code": True,
        "low_cpu_mem_usage": True,
    }
    
    # Add Flash Attention if enabled
    if USE_FLASH_ATTENTION:
        model_kwargs["attn_implementation"] = ATTN_IMPLEMENTATION
        print("✓ Flash Attention 2 enabled")
    
    # Add memory constraints for multi-GPU
    if max_memory:
        model_kwargs["max_memory"] = max_memory
        print(f"✓ Memory allocation: {GPU_MEMORY_PER_DEVICE} per GPU")
    
    model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)
    print("✓ Model loaded on GPU(s)")

# Ensure padding token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

# Enable padding on the left for batch generation
tokenizer.padding_side = "left"

# Report device placement
try:
    if hasattr(model, 'hf_device_map'):
        devices_used = set(str(v) for v in model.hf_device_map.values())
        print(f"Device placement: Model split across {devices_used}")
    else:
        print(f"Device: {next(model.parameters()).device}")
except Exception:
    pass

print(f"Model size: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")
print("✓ Model ready for inference")

Loading SFT model from Hugging Face...
⚠ Loading model on CPU (FORCE_CPU=True)


/home/imane/gemma_env/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/imane/gemma_env/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/imane/gemma_env/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 5/5 [00:20<00:00,  4.03s/it]



✓ Model loaded on CPU
Device placement: {'': 'cpu'}
Model size: 9.24B parameters
✓ Model ready for inference


## Load Training Data

In [ ]:
# ===========================
# LOAD AND SAMPLE 100K FROM YOUR DATA
# ===========================

print("Loading source data for synthetic data generation...")
print("Note: We don't need reference translations - just source texts\n")

# Try loading from multiple possible sources
all_data = []

# 1. Try loading from test_prompts.jsonl
try:
    test_data = load_test_prompts(TEST_PROMPTS)
    all_data.extend(test_data)
    print(f"✓ Loaded {len(test_data)} prompts from test_prompts.jsonl")
except FileNotFoundError:
    print("⚠ test_prompts.jsonl not found")

# 2. Try loading from english_inputs.json (your large file)
english_inputs_path = PROJECT_DIR / "english_inputs.json"
if english_inputs_path.exists():
    try:
        import json
        with open(english_inputs_path, 'r', encoding='utf-8') as f:
            english_data = json.load(f)
        
        # Convert to our format
        if isinstance(english_data, list):
            for item in english_data:
                if isinstance(item, str):
                    all_data.append({'source': item, 'source_lang': 'en'})
                elif isinstance(item, dict):
                    text = item.get('text', item.get('source', item.get('sentence', '')))
                    if text:
                        all_data.append({'source': text, 'source_lang': 'en'})
        print(f"✓ Loaded {len(english_data)} English inputs")
    except Exception as e:
        print(f"⚠ Error loading english_inputs.json: {e}")

# 3. Try loading from french_inputs.json
french_inputs_path = PROJECT_DIR / "french_inputs.json"
if french_inputs_path.exists():
    try:
        with open(french_inputs_path, 'r', encoding='utf-8') as f:
            french_data = json.load(f)
        
        if isinstance(french_data, list):
            for item in french_data:
                if isinstance(item, str):
                    all_data.append({'source': item, 'source_lang': 'fr'})
                elif isinstance(item, dict):
                    text = item.get('text', item.get('source', item.get('sentence', '')))
                    if text:
                        all_data.append({'source': text, 'source_lang': 'fr'})
        print(f"✓ Loaded {len(french_data)} French inputs")
    except Exception as e:
        print(f"⚠ Error loading french_inputs.json: {e}")

print(f"\nTotal available data: {len(all_data):,} samples")

# ===========================
# SAMPLE 100K FOR TRAINING
# ===========================

SAMPLE_SIZE = 100_000  # Sample 100K as requested

# Shuffle and sample
random.shuffle(all_data)
training_samples = all_data[:min(SAMPLE_SIZE, len(all_data))]

print(f"\n🎯 Sampled {len(training_samples):,} samples for synthetic data generation")
print(f"   (from {len(all_data):,} total available)")

# Show sample distribution
en_count = sum(1 for s in training_samples if s['source_lang'] == 'en')
fr_count = sum(1 for s in training_samples if s['source_lang'] == 'fr')
print(f"\n   Language distribution:")
print(f"   - English: {en_count:,} ({100*en_count/len(training_samples):.1f}%)")
print(f"   - French: {fr_count:,} ({100*fr_count/len(training_samples):.1f}%)")

Loading test prompts for synthetic data generation...
Note: We don't need reference translations - just source texts

Loaded 91 test prompts from file

Using 91 samples for synthetic data generation


## Generate Translation Candidates

In [ ]:
# ===========================
# DUAL-GPU PARALLEL GENERATION FOR 100K SAMPLES
# ===========================

import torch.multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import threading
import queue

# Configuration for dual 5090 setup
MEGA_BATCH_SIZE = 32  # Per GPU batch size
NUM_CANDIDATES = 4    # Candidates per source
MAX_NEW_TOKENS = 128  # Shorter for speed

print(f"🚀 Dual-GPU Parallel Generation Configuration:")
print(f"   GPUs available: {NUM_GPUS}")
print(f"   Batch size per GPU: {MEGA_BATCH_SIZE}")
print(f"   Total batch size: {MEGA_BATCH_SIZE * max(NUM_GPUS, 1)}")
print(f"   Candidates per source: {NUM_CANDIDATES}")


def generate_batch_on_device(batch_sources, batch_langs, num_candidates=4):
    """
    Generate candidates for a batch - model handles multi-GPU automatically
    with device_map="auto"
    """
    prompts = [format_translation_prompt(src, lang) 
               for src, lang in zip(batch_sources, batch_langs)]
    
    batch_size = len(prompts)
    
    # Tokenize
    inputs = tokenizer(
        prompts, 
        return_tensors="pt", 
        padding=True, 
        truncation=True,
        max_length=256
    ).to(model.device)
    
    # Generate with automatic multi-GPU handling
    with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=0.9,
            top_p=0.95,
            do_sample=True,
            num_return_sequences=num_candidates,
            num_beams=1,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # Decode all at once
    all_decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # Reshape results
    all_candidates = []
    for i in range(batch_size):
        candidates = []
        for j in range(num_candidates):
            idx = i * num_candidates + j
            if idx < len(all_decoded):
                full_text = all_decoded[idx]
                translation = full_text.split("Arabic translation:")[-1].strip()
                candidates.append({
                    'translation': translation,
                    'config': {'temperature': 0.9, 'top_p': 0.95}
                })
        all_candidates.append(candidates)
    
    return all_candidates


def score_batch_fast(sources, candidates_batch):
    """
    Fast scoring using heuristics (skip COMET for speed)
    """
    all_scores = []
    
    for source, candidates in zip(sources, candidates_batch):
        scored = []
        src_len = len(source.split())
        
        for cand in candidates:
            trans = cand['translation']
            tgt_len = len(trans.split())
            
            # Fast heuristic scores
            # 1. Length ratio
            length_score = min(tgt_len / max(src_len * 1.2, 1), 1.5) / 1.5
            
            # 2. Arabic character ratio
            arabic_chars = sum(1 for c in trans if '\u0600' <= c <= '\u06FF')
            total_chars = len(trans.replace(' ', '')) + 1
            arabic_score = min(arabic_chars / total_chars, 1.0)
            
            # 3. Diversity (unique words ratio)
            words = trans.split()
            diversity = len(set(words)) / max(len(words), 1)
            
            # 4. Reasonable length penalty
            length_penalty = 1.0 if 3 < tgt_len < 150 else 0.5
            
            # Combined score
            combined = (
                arabic_score * 0.5 +      # Most important: is it Arabic?
                length_score * 0.2 +
                diversity * 0.2 +
                length_penalty * 0.1
            )
            
            scored.append({
                'translation': trans,
                'scores': {
                    'arabic': arabic_score,
                    'length': length_score,
                    'diversity': diversity,
                    'combined': combined
                },
                'config': cand['config']
            })
        
        all_scores.append(scored)
    
    return all_scores


# Assign main generation function
generate_candidates = generate_batch_on_device
score_translations_batched = score_batch_fast

# Quick test
print("\n🧪 Testing generation pipeline...")
if len(training_samples) >= 2:
    test_batch = training_samples[:2]
    test_sources = [s['source'] for s in test_batch]
    test_langs = [s['source_lang'] for s in test_batch]
    
    import time
    start = time.time()
    test_result = generate_candidates(test_sources, test_langs, num_candidates=NUM_CANDIDATES)
    elapsed = time.time() - start
    
    print(f"✓ Generated {len(test_result[0])} candidates for {len(test_batch)} sources in {elapsed:.2f}s")
    print(f"   Throughput: {len(test_batch)/elapsed:.1f} sentences/second")
    print(f"   Sample output: {test_result[0][0]['translation'][:80]}...")
else:
    print("⚠ Not enough samples for testing")

Testing with: Whether organize files will group directories according to their filetype....

Generated 8 candidates:
1. سواء كانت تنظيم الملفات ستجمع الدلائل وفقًا لنوع الملف....
2. سوف تُعزز ملفات التنظيم المجلدات وفقًا لنوع ملفها....
3. سواء كانت تنظيم الملفات ستجمع الدلائل حسب نوع الملف....
4. سيتم جمع الملفات حسب نوع الملف....
5. سواء كان تصنيف الملفات سيتم فيه تجميع الدلائل وفقًا لنوع الملف....
6. سواءً كانت الملفات المنظمة ستُجمع الدلائل وفقًا لنوع ملفاتها....
7. سوف تمثل ملفات التنظيم المجموعات التجهيزية وفقاً لنوعها....
8. سوف يجمع تنظيم الملفات المجلدات وفقًا لنوع الملف الخاص بها....

Generated 8 candidates:
1. سواء كانت تنظيم الملفات ستجمع الدلائل وفقًا لنوع الملف....
2. سوف تُعزز ملفات التنظيم المجلدات وفقًا لنوع ملفها....
3. سواء كانت تنظيم الملفات ستجمع الدلائل حسب نوع الملف....
4. سيتم جمع الملفات حسب نوع الملف....
5. سواء كان تصنيف الملفات سيتم فيه تجميع الدلائل وفقًا لنوع الملف....
6. سواءً كانت الملفات المنظمة ستُجمع الدلائل وفقًا لنوع ملفاتها....
7. سوف تمثل ملفات الت

## Alternative: vLLM for Maximum Speed (Optional)

If you have vLLM installed (`pip install vllm`), uncomment and run the cell below for 3-10x faster generation.

In [ ]:
# ===========================
# OPTIONAL: vLLM FOR 3-10X FASTER GENERATION
# ===========================
# Set USE_VLLM = True if you have vLLM installed: pip install vllm

USE_VLLM = False  # Set to True to use vLLM instead of transformers

if USE_VLLM:
    try:
        from vllm import LLM, SamplingParams
        
        print("🚀 Loading model with vLLM (tensor parallel across 2 GPUs)...")
        
        # Unload transformers model to free memory
        del model
        torch.cuda.empty_cache()
        
        # Load with vLLM - automatically uses both GPUs
        vllm_model = LLM(
            model="ModelSpace/GemmaX2-28-9B-v0.1",
            tensor_parallel_size=NUM_GPUS,  # Use both GPUs
            dtype="bfloat16",
            max_model_len=512,
            trust_remote_code=True,
        )
        
        vllm_sampling = SamplingParams(
            temperature=0.9,
            top_p=0.95,
            max_tokens=MAX_NEW_TOKENS,
            n=NUM_CANDIDATES  # Generate N candidates per prompt
        )
        
        def generate_with_vllm(batch_sources, batch_langs, num_candidates=4):
            """Ultra-fast generation with vLLM"""
            prompts = [format_translation_prompt(src, lang) 
                       for src, lang in zip(batch_sources, batch_langs)]
            
            # vLLM handles batching internally - very efficient
            outputs = vllm_model.generate(prompts, vllm_sampling)
            
            all_candidates = []
            for output in outputs:
                candidates = []
                for completion in output.outputs:
                    text = completion.text
                    if "Arabic translation:" in text:
                        translation = text.split("Arabic translation:")[-1].strip()
                    else:
                        translation = text.strip()
                    candidates.append({
                        'translation': translation,
                        'config': {'temperature': 0.9, 'top_p': 0.95}
                    })
                all_candidates.append(candidates)
            
            return all_candidates
        
        # Replace generation function with vLLM version
        generate_candidates = generate_with_vllm
        print("✓ vLLM loaded - using tensor parallelism across 2 GPUs")
        print("   Expected speedup: 3-10x faster than transformers")
        
    except ImportError:
        print("⚠ vLLM not installed. Using transformers instead.")
        print("   Install with: pip install vllm")
    except Exception as e:
        print(f"⚠ vLLM failed to load: {e}")
        print("   Falling back to transformers")
else:
    print("ℹ Using transformers for generation")
    print("   Set USE_VLLM=True and install vllm for 3-10x faster generation")

Loading evaluation metrics...


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 34267.19it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
Encoder model frozen.


✓ COMET loaded
✓ CHRF loaded
✓ BERTScore (will load on first use)

All metrics ready!


/home/imane/gemma_env/lib/python3.12/site-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


## Score Translations

In [ ]:
# ===========================
# COMET-Based Translation Scoring
# ===========================

# Batch size for COMET scoring (adjust based on GPU memory)
COMET_BATCH_SIZE = 64  # Can increase if GPU has headroom

# Determine which GPU to use for COMET (avoid conflict with translation model)
if NUM_GPUS > 1:
    COMET_GPU_ID = 1  # Use second GPU for COMET
else:
    COMET_GPU_ID = 0 if torch.cuda.is_available() else -1  # -1 means CPU

print(f"📊 COMET Scoring Configuration:")
print(f"   COMET available: {COMET_AVAILABLE}")
print(f"   COMET GPU: {COMET_GPU_ID if COMET_GPU_ID >= 0 else 'CPU'}")
print(f"   Batch size: {COMET_BATCH_SIZE}")


def score_translation_single(source: str, translation: str) -> dict:
    """Score a single translation using COMET-QE + heuristics.
    
    Returns dict with 'comet', 'length_score', 'arabic_score', 'combined'.
    """
    scores = {}
    
    # 1. COMET-QE Score (main quality metric)
    if COMET_AVAILABLE and comet_model is not None:
        try:
            comet_input = [{'src': source, 'mt': translation}]
            gpus = 1 if COMET_GPU_ID >= 0 else 0
            comet_output = comet_model.predict(comet_input, batch_size=1, gpus=gpus)
            scores['comet'] = float(comet_output.scores[0])
        except Exception as e:
            print(f"⚠️ COMET scoring failed: {e}")
            scores['comet'] = 0.5  # Neutral fallback
    else:
        scores['comet'] = 0.5
    
    # 2. Length ratio score
    src_len = len(source.split())
    tgt_len = len(translation.split())
    length_ratio = tgt_len / max(src_len, 1)
    if 0.6 <= length_ratio <= 1.5:
        scores['length_score'] = 1.0
    elif 0.4 <= length_ratio <= 2.0:
        scores['length_score'] = 0.7
    else:
        scores['length_score'] = 0.3
    
    # 3. Arabic character ratio (should be mostly Arabic)
    arabic_chars = sum(1 for c in translation if '\u0600' <= c <= '\u06FF')
    total_chars = len(translation.replace(' ', ''))
    scores['arabic_score'] = arabic_chars / max(total_chars, 1)
    
    # 4. Combined weighted score (COMET is primary)
    scores['combined'] = (
        scores['comet'] * 0.7 +
        scores['length_score'] * 0.15 +
        scores['arabic_score'] * 0.15
    )
    
    return scores


def score_translations_batch(sources: list, translations: list) -> list:
    """Score multiple translations efficiently using batched COMET.
    
    Args:
        sources: List of source sentences
        translations: List of corresponding translations
        
    Returns:
        List of score dicts
    """
    assert len(sources) == len(translations), "Sources and translations must match"
    
    all_scores = []
    
    # Prepare COMET inputs
    if COMET_AVAILABLE and comet_model is not None:
        comet_inputs = [{'src': s, 'mt': t} for s, t in zip(sources, translations)]
        
        # Only print for large batches to avoid spam
        if len(comet_inputs) > 10:
            print(f"🔄 Scoring {len(comet_inputs)} translations with COMET...")
        
        try:
            # Use the configured GPU for COMET
            gpus = 1 if COMET_GPU_ID >= 0 else 0
            comet_output = comet_model.predict(
                comet_inputs, 
                batch_size=COMET_BATCH_SIZE,
                gpus=gpus,
                progress_bar=(len(comet_inputs) > 100)  # Only show progress for large batches
            )
            comet_scores = comet_output.scores
            
            if len(comet_inputs) > 10:
                avg_score = sum(comet_scores) / len(comet_scores)
                print(f"✅ COMET complete. Mean: {avg_score:.4f}, Range: [{min(comet_scores):.4f}, {max(comet_scores):.4f}]")
        except Exception as e:
            print(f"⚠️ Batch COMET failed: {e}, using fallbacks")
            comet_scores = [0.5] * len(translations)
    else:
        if len(sources) > 10:
            print("⚠️ COMET not available, using heuristic fallbacks")
        comet_scores = [0.5] * len(translations)
    
    # Calculate all scores
    for i, (src, tgt) in enumerate(zip(sources, translations)):
        scores = {}
        scores['comet'] = float(comet_scores[i])
        
        # Length ratio
        src_len = len(src.split())
        tgt_len = len(tgt.split())
        length_ratio = tgt_len / max(src_len, 1)
        if 0.6 <= length_ratio <= 1.5:
            scores['length_score'] = 1.0
        elif 0.4 <= length_ratio <= 2.0:
            scores['length_score'] = 0.7
        else:
            scores['length_score'] = 0.3
        
        # Arabic ratio
        arabic_chars = sum(1 for c in tgt if '\u0600' <= c <= '\u06FF')
        total_chars = len(tgt.replace(' ', ''))
        scores['arabic_score'] = arabic_chars / max(total_chars, 1)
        
        # Combined
        scores['combined'] = (
            scores['comet'] * 0.7 +
            scores['length_score'] * 0.15 +
            scores['arabic_score'] * 0.15
        )
        
        all_scores.append(scores)
    
    return all_scores


# Quick test
print("\n" + "="*50)
print("Testing COMET scoring...")
print("="*50)

if 'test_result' in dir() and len(test_result) > 0 and len(test_result[0]) > 0:
    test_src = training_samples[0]['source']
    test_tgt = test_result[0][0]['translation']
    
    print(f"Source: {test_src[:80]}...")
    print(f"Translation: {test_tgt[:80]}...")
    
    test_scores = score_translation_single(test_src, test_tgt)
    print("\nScores:")
    for k, v in test_scores.items():
        print(f"  {k}: {v:.4f}")
    
    # Verify COMET is working
    if COMET_AVAILABLE and test_scores['comet'] != 0.5:
        print("\n✅ COMET is working correctly!")
    elif COMET_AVAILABLE:
        print("\n⚠️ COMET returned default score - may indicate an issue")
    else:
        print("\n⚠️ COMET not available - using heuristics only")
else:
    print("⚠️ No test candidates available yet - run generation cell first")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
You are using a CUDA device ('NVIDIA GeForce RTX 5090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
You are using a CUDA device ('NVIDIA GeForce RTX 5090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For

Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Scores: {'comet': '0.0000', 'length_score': '0.9000', 'arabic_score': '1.0000', 'combined': '0.2800'}


## Generate Synthetic Preference Dataset

In [ ]:
def create_preference_pairs(candidates_with_scores):
    """Create pairwise preference data from scored candidates"""
    preference_pairs = []
    sorted_candidates = sorted(
        candidates_with_scores, 
        key=lambda x: x['scores']['combined'], 
        reverse=True
    )
    
    # Create pairs: best vs worst for efficiency
    if len(sorted_candidates) >= 2:
        # Top candidates vs bottom candidates
        top_n = min(2, len(sorted_candidates))
        bottom_start = max(0, len(sorted_candidates) - 2)
        
        for i in range(top_n):
            for j in range(bottom_start, len(sorted_candidates)):
                if i < j:
                    chosen = sorted_candidates[i]
                    rejected = sorted_candidates[j]
                    score_diff = chosen['scores']['combined'] - rejected['scores']['combined']
                    
                    if score_diff > 0.02:  # Minimum margin
                        preference_pairs.append({
                            'chosen': chosen['translation'],
                            'rejected': rejected['translation'],
                            'chosen_score': chosen['scores']['combined'],
                            'rejected_score': rejected['scores']['combined'],
                            'score_margin': score_diff
                        })
    
    return preference_pairs


def score_batch_candidates(batch_sources: list, batch_candidates: list) -> list:
    """Score all candidates in a batch using COMET.
    
    Args:
        batch_sources: List of source sentences
        batch_candidates: List of lists, each containing candidate translations
        
    Returns:
        List of lists with scored candidates
    """
    # Flatten all candidates for batch scoring
    all_sources = []
    all_translations = []
    indices = []  # Track which batch/candidate each score belongs to
    
    for batch_idx, (source, candidates) in enumerate(zip(batch_sources, batch_candidates)):
        for cand_idx, candidate in enumerate(candidates):
            all_sources.append(source)
            all_translations.append(candidate['translation'])
            indices.append((batch_idx, cand_idx))
    
    # Batch score with COMET
    all_scores = score_translations_batch(all_sources, all_translations)
    
    # Reconstruct structure
    scored_batch = [[] for _ in range(len(batch_sources))]
    for (batch_idx, cand_idx), scores in zip(indices, all_scores):
        candidate = batch_candidates[batch_idx][cand_idx]
        scored_batch[batch_idx].append({
            'translation': candidate['translation'],
            'config': candidate.get('config', {}),
            'scores': scores
        })
    
    return scored_batch


# ===========================
# MAIN GENERATION LOOP - DUAL GPU with COMET Scoring
# ===========================

print("\n" + "=" * 80)
print("🚀 DUAL-GPU PARALLEL SYNTHETIC DATA GENERATION")
print("   with COMET Quality Scoring")
print("=" * 80)
print(f"\nConfiguration:")
print(f"   Total samples: {len(training_samples):,}")
print(f"   Batch size: {MEGA_BATCH_SIZE}")
print(f"   Candidates per source: {NUM_CANDIDATES}")
print(f"   GPUs: {NUM_GPUS}")
print(f"   COMET enabled: {COMET_AVAILABLE}")
print(f"   COMET batch size: {COMET_BATCH_SIZE}")

# Estimate time (COMET adds ~2-3 seconds per batch)
estimated_rate = 8 if COMET_AVAILABLE else 15  # samples/second with COMET
estimated_hours = len(training_samples) / estimated_rate / 3600
print(f"\n⏱️ Estimated time: {estimated_hours:.1f} hours")
print("=" * 80)

synthetic_dataset = []
candidates_path = DATA_DIR / "generated_candidates.jsonl"

import time
start_time = time.time()
samples_processed = 0
errors_count = 0
comet_scores_collected = []

# Calculate number of batches
num_batches = (len(training_samples) + MEGA_BATCH_SIZE - 1) // MEGA_BATCH_SIZE

# Open output file for streaming writes
with open(candidates_path, 'w', encoding='utf-8') as cand_f:
    
    for batch_idx in tqdm(range(num_batches), desc="Processing batches"):
        batch_start = time.time()
        
        # Get batch
        start_idx = batch_idx * MEGA_BATCH_SIZE
        end_idx = min(start_idx + MEGA_BATCH_SIZE, len(training_samples))
        batch_samples = training_samples[start_idx:end_idx]
        
        batch_sources = [s['source'] for s in batch_samples]
        batch_langs = [s['source_lang'] for s in batch_samples]
        
        try:
            # Step 1: Generate candidates
            batch_candidates = generate_candidates(
                batch_sources,
                batch_langs,
                num_candidates=NUM_CANDIDATES
            )
            
            # Step 2: Score candidates with COMET
            scored_batch = score_batch_candidates(batch_sources, batch_candidates)
            
            # Process each sample in batch
            for sample, candidates, scored in zip(batch_samples, batch_candidates, scored_batch):
                # Save raw candidates
                cand_record = {
                    'source': sample['source'],
                    'source_lang': sample['source_lang'],
                    'candidates': [c['translation'] for c in candidates]
                }
                cand_f.write(json.dumps(cand_record, ensure_ascii=False) + '\n')
                
                # Collect COMET scores for statistics
                for s in scored:
                    comet_scores_collected.append(s['scores']['comet'])
                
                # Create preference pairs
                pairs = create_preference_pairs(scored)
                for pair in pairs:
                    synthetic_dataset.append({
                        'source': sample['source'],
                        'source_lang': sample['source_lang'],
                        'chosen': pair['chosen'],
                        'rejected': pair['rejected'],
                        'chosen_score': pair['chosen_score'],
                        'rejected_score': pair['rejected_score'],
                        'margin': pair['score_margin']
                    })
            
            samples_processed += len(batch_samples)
            
            # Clear GPU cache periodically
            if batch_idx % 50 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
                
        except Exception as e:
            errors_count += 1
            if errors_count <= 5:
                print(f"\n⚠️ Error in batch {batch_idx}: {e}")
                import traceback
                traceback.print_exc()
            continue
        
        # Progress update every 50 batches
        if (batch_idx + 1) % 50 == 0:
            elapsed = time.time() - start_time
            rate = samples_processed / elapsed if elapsed > 0 else 0
            remaining = (len(training_samples) - samples_processed) / rate if rate > 0 else 0
            
            # COMET stats
            if comet_scores_collected:
                avg_comet = sum(comet_scores_collected) / len(comet_scores_collected)
                min_comet = min(comet_scores_collected)
                max_comet = max(comet_scores_collected)
            else:
                avg_comet = min_comet = max_comet = 0
            
            # GPU memory info
            if torch.cuda.is_available():
                gpu_mem = torch.cuda.memory_allocated() / 1e9
                gpu_mem_max = torch.cuda.max_memory_allocated() / 1e9
            else:
                gpu_mem = gpu_mem_max = 0
            
            print(f"\n📊 Progress Report:")
            print(f"   Samples: {samples_processed:,}/{len(training_samples):,} ({100*samples_processed/len(training_samples):.1f}%)")
            print(f"   Pairs generated: {len(synthetic_dataset):,}")
            print(f"   COMET scores: avg={avg_comet:.4f}, range=[{min_comet:.4f}, {max_comet:.4f}]")
            print(f"   Rate: {rate:.1f} samples/sec")
            print(f"   ETA: {remaining/3600:.2f} hours")
            print(f"   GPU Memory: {gpu_mem:.1f}GB (peak: {gpu_mem_max:.1f}GB)")
            print(f"   Errors: {errors_count}")
        
        # Save checkpoint every 500 batches
        if (batch_idx + 1) % 500 == 0:
            checkpoint_path = DATA_DIR / f"synthetic_preferences_checkpoint.jsonl"
            with open(checkpoint_path, 'w', encoding='utf-8') as f:
                for item in synthetic_dataset:
                    f.write(json.dumps(item, ensure_ascii=False) + '\n')
            print(f"   💾 Checkpoint saved ({len(synthetic_dataset):,} pairs)")
        
        # Flush output
        cand_f.flush()

# Final stats
total_time = time.time() - start_time
print("\n" + "=" * 80)
print("✅ GENERATION COMPLETE!")
print("=" * 80)
print(f"   Samples processed: {samples_processed:,}")
print(f"   Preference pairs: {len(synthetic_dataset):,}")
print(f"   Total time: {total_time/3600:.2f} hours")
print(f"   Average rate: {samples_processed/total_time:.1f} samples/sec")
if comet_scores_collected:
    print(f"   COMET scores: avg={sum(comet_scores_collected)/len(comet_scores_collected):.4f}")
print(f"   Errors: {errors_count}")

Generating synthetic preference dataset...
Processing 91 samples...

Saving generated candidates to /home/imane/Downloads/Reinforcement Learning/data/generated_candidates.jsonl


Generating preferences:   0%|          | 0/91 [00:00<?, ?it/s]💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU c

Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



...

Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



Generating preferences:  21%|██        | 19/91 [59:05<3:10:50, 159.04s/it]

Search for `cudaErrorNoKernelImageForDevice' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



## Save Synthetic Dataset

In [ ]:
# Save to JSONL format
print(f"Saving dataset to {SYNTHETIC_PREFERENCES}...")

with open(SYNTHETIC_PREFERENCES, 'w', encoding='utf-8') as f:
    for item in synthetic_dataset:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"✓ Saved {len(synthetic_dataset)} preference pairs")

# Save summary statistics
stats = {
    'total_pairs': len(synthetic_dataset),
    'avg_margin': sum(item['margin'] for item in synthetic_dataset) / len(synthetic_dataset),
    'avg_chosen_score': sum(item['chosen_score'] for item in synthetic_dataset) / len(synthetic_dataset),
    'avg_rejected_score': sum(item['rejected_score'] for item in synthetic_dataset) / len(synthetic_dataset),
    'num_sources': len(training_samples)
}

stats_path = DATA_DIR / "synthetic_data_stats.json"
with open(stats_path, 'w') as f:
    json.dump(stats, f, indent=2)

print(f"\nDataset Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value if isinstance(value, int) else f'{value:.4f}'}")

## Sample Preference Pairs

In [ ]:
# Display some examples
print("Sample preference pairs:\n")
print("=" * 80)

for i, item in enumerate(random.sample(synthetic_dataset, min(3, len(synthetic_dataset))), 1):
    print(f"\nExample {i}:")
    print(f"Source ({item['source_lang']}): {item['source'][:150]}")
    print(f"\nChosen (score: {item['chosen_score']:.3f}): {item['chosen'][:150]}")
    print(f"\nRejected (score: {item['rejected_score']:.3f}): {item['rejected'][:150]}")
    print(f"\nMargin: {item['margin']:.3f}")
    print("=" * 80)

## Next Step

Proceed to **notebook 2** to train the reward model using this synthetic preference data.

## Quick Process: Convert Partial Data to Preferences

Run this cell to process the candidates you already generated (without re-running generation).